In [1]:
# Increasing Contrast to the Images
from PIL import ImageFile, ImageFilter
import numpy as np
ImageFile.LOAD_TRUNCATED_IMAGES = True
import shutil
import pandas as pd
import os
from datasets import load_dataset
import cv2 as cv
dataset = load_dataset("yh0701/FracAtlas_dataset", trust_remote_code=True)

C:\Users\LEGION\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def create_directories(directory_paths):
    for path in directory_paths.values():
        os.makedirs(path, exist_ok=True)

In [4]:
# Dataprep for CLAHE Detection
for j in ['train', 'test', 'validation']:
    for i, item in enumerate(dataset[j]):
        image_data = np.array(item['image'])
        clahe = cv.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
        if len(image_data.shape) == 3 and image_data.shape[2]>1:
            image_data = cv.cvtColor(image_data, cv.COLOR_RGB2GRAY)
            equ = clahe.apply(image_data)
        else:
            equ = clahe.apply(image_data)
        image_id = item['image_id']
        image_path = f'CLAHE\{image_id}'
        cv.imwrite(image_path, equ)

In [5]:
# Distribute CLAHE Images to Cooresponding Folders
train_csv = "FracAtlas/FracAtlas/Utilities/Fracture Split/train.csv"
valid_csv = "FracAtlas/FracAtlas/Utilities/Fracture Split/valid.csv"
test_csv = "FracAtlas/FracAtlas/Utilities/Fracture Split/test.csv"

source_dir = "CLAHE"

dest_dirs = {
    'train': "CLAHE_Training_Detection/dataset/train/images",
    'valid': "CLAHE_Training_Detection/dataset/valid/images",
    'test': "CLAHE_Training_Detection/dataset/test/images", 
}

def copy_images(csv_file, target_dir):
    df = pd.read_csv(csv_file)
    os.makedirs(target_dir, exist_ok=True)
    for image_name in df['image_id']:
        src_path = os.path.join(source_dir, image_name)
        dst_path = os.path.join(target_dir, image_name)
        shutil.copy(src_path, dst_path)

copy_images(train_csv, dest_dirs['train'])
copy_images(valid_csv, dest_dirs['valid'])
copy_images(test_csv, dest_dirs['test'])

In [6]:
# Distribute Labels to Cooresponding Folders
source_dir = "FracAtlas/FracAtlas/Annotations/YOLO"

dest_dirs = {
    'train': "D:/project2/CLAHE_Training_Detection/dataset/train/labels",
    'valid': "D:/project2/CLAHE_Training_Detection/dataset/valid/labels",
    'test': "D:/project2/CLAHE_Training_Detection/dataset/test/labels", 
}
create_directories(dest_dirs)

def copy_labels(csv_file, target_dir):
    df = pd.read_csv(csv_file)
    for image_name in df['image_id']:
        label_name = image_name.replace('.jpg', '.txt')
        shutil.copy(os.path.join(source_dir, label_name), os.path.join(target_dir, label_name))

copy_labels(train_csv, dest_dirs['train'])
copy_labels(valid_csv, dest_dirs['valid'])
copy_labels(test_csv, dest_dirs['test'])

In [8]:
# Distribute CLAHE Images for Segmentation
source_dir = "CLAHE"

dest_dirs = {
    'train': "CLAHE_Training_Seg/dataset/train/images",
    'valid': "CLAHE_Training_Seg/dataset/valid/images",
    'test': "CLAHE_Training_Seg/dataset/test/images", 
}

def copy_images(csv_file, target_dir):
    df = pd.read_csv(csv_file)
    os.makedirs(target_dir, exist_ok=True)
    for image_name in df['image_id']:
        src_path = os.path.join(source_dir, image_name)
        dst_path = os.path.join(target_dir, image_name)
        shutil.copy(src_path, dst_path)

copy_images(train_csv, dest_dirs['train'])
copy_images(valid_csv, dest_dirs['valid'])
copy_images(test_csv, dest_dirs['test'])

In [7]:
# Distribute Labels for Segmentation
source_dir = "YOLO_Segmentation_Labels"

dest_dirs = {
    'train': "CLAHE_Training_Seg/dataset/train/labels",
    'valid': "CLAHE_Training_Seg/dataset/valid/labels",
    'test': "CLAHE_Training_Seg/dataset/test/labels", 
}
create_directories(dest_dirs)

def copy_labels(csv_file, target_dir):
    df = pd.read_csv(csv_file)
    for image_name in df['image_id']:
        label_name = image_name.replace('.jpg', '.txt')
        shutil.copy(os.path.join(source_dir, label_name), os.path.join(target_dir, label_name))

copy_labels(train_csv, dest_dirs['train'])
copy_labels(valid_csv, dest_dirs['valid'])
copy_labels(test_csv, dest_dirs['test'])